<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/LandCover_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 81.5 MB/s eta 0:00:00


In [3]:
import os
import gdown
import zipfile
import rasterio
from skimage.transform import resize
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import random
from keras.utils import to_categorical

In [ ]:
# Download the dataset
url = 'https://drive.google.com/uc?id=1f4eGmykyiczmNz2VPeNNmQ7aC7q8N_hD'
output = '/content/california_land_use.zip'
gdown.download(url, output, quiet=False)

# Extract the dataset
cwd = os.getcwd()
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(cwd+'/sample_data')


Downloading...
From: https://drive.google.com/uc?id=1f4eGmykyiczmNz2VPeNNmQ7aC7q8N_hD
To: /content/california_land_use.zip
 52%|█████▏    | 1.44G/2.76G [00:20<00:16, 81.3MB/s]

In [ ]:
# Required libraries
import rasterio
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import glob

In [ ]:
# Loading data from geotiff files
def load_data(directory):
    data = []
    files = glob.glob(os.path.join(directory, "*.tif"))
    for file in files:
        with rasterio.open(file) as src:
            band_data = []
            for band in src.read():
                band_data.append(band)
            data.append(np.dstack(band_data))
    
    return np.array(data)

In [ ]:
def load_and_reshape_image(image_path, img_height, img_width):
    with rasterio.open(image_path) as src:
        # Read the image data
        image = src.read()
        # Reshape the image
        image = image.transpose((1, 2, 0))
        # Resize image if necessary
        if image.shape[0] != img_height or image.shape[1] != img_width:
            image = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
        # Ensure labels do not have an extra channel dimension
        if len(image.shape) == 3 and image.shape[2] == 1:
            image = np.squeeze(image, axis=2)
        return image

In [ ]:
def load_images_and_labels(image_files, label_files, img_height, img_width, num_classes):
  images = []
  labels = []

  image_files = glob.glob(os.path.join(image_files, "*.tif"))
  for image_file in image_files:
      image = load_and_reshape_image(image_file, img_height, img_width)
      images.append(image)

  label_files = glob.glob(os.path.join(label_files, "*.tif"))
  for label_file in label_files:
      label = load_and_reshape_image(label_file, img_height, img_width)
      label -= 1  # adjust labels to be in the range 0-8 instead of 1-9
      label = to_categorical(label, num_classes=9)   # one-hot encode the labels
      labels.append(label)

  return np.array(images), np.array(labels)



In [140]:
X, y = load_images_and_labels('/content/sample_data/training/rgbNIR/', '/content/sample_data/training/labels/', 640, 640,num_classes =9)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [121]:
# # Model parameters
# img_height = X_train.shape[1]  # replace these with the dimensions of your images
# img_width = X_train.shape[2]
# n_bands = X_train.shape[3]
# output_channels = y_train.shape[3]


In [122]:
# n_bands

In [123]:
# img_size + (3,)

In [146]:
def create_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size)  # Change the number of channels to 4

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])  # Use sparse categorical cross-entropy loss
    return model


In [147]:
model = create_model(img_size = (640,640,3),num_classes =9)

In [148]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=50)

Epoch 1/50
6/6 [==============================] - 17s 504ms/step - loss: 4.4731 - accuracy: 0.5446 - val_loss: 47.2190 - val_accuracy: 0.0124
Epoch 2/50
6/6 [==============================] - 2s 284ms/step - loss: 1.8402 - accuracy: 0.6979 - val_loss: 134.4627 - val_accuracy: 0.0025
Epoch 3/50
6/6 [==============================] - 2s 293ms/step - loss: 1.2094 - accuracy: 0.6715 - val_loss: 7.4014 - val_accuracy: 0.2286
Epoch 4/50
6/6 [==============================] - 2s 285ms/step - loss: 0.9126 - accuracy: 0.7334 - val_loss: 8.1328 - val_accuracy: 0.1433
Epoch 5/50
6/6 [==============================] - 2s 279ms/step - loss: 0.7436 - accuracy: 0.7515 - val_loss: 3.3124 - val_accuracy: 0.6958
Epoch 6/50
6/6 [==============================] - 2s 278ms/step - loss: 0.7871 - accuracy: 0.7323 - val_loss: 1.9742 - val_accuracy: 0.6929
Epoch 7/50
6/6 [==============================] - 2s 279ms/step - loss: 0.7623 - accuracy: 0.7487 - val_loss: 1.9313 - val_accuracy: 0.7002
Epoch 8/50
6/6 [